In [4]:
import pandas as pd
import os
import time
import yaml
from datetime import datetime, timedelta
from tqdm.notebook import tqdm  # 使用专门为 Jupyter 优化的进度条
from WindPy import w

# 自动重载修改后的模块（如果你以后把类存入 .py 文件）
%load_ext autoreload
%autoreload 2

In [5]:
CONFIG_PATH = "config.yaml"

def load_config(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"未找到配置文件: {path}")
    with open(path, 'r', encoding='utf-8') as f:
        return yaml.safe_load(f)

cfg = load_config(CONFIG_PATH)
print(f"✅ 配置已加载，目标路径: {cfg['paths']['raw_data']}")

✅ 配置已加载，目标路径: data/raw_parquet


In [9]:
def validate_config(config):
    """
    检查配置字典是否包含所有必需的键和有效的路径
    """
    required_structure = {
        'paths': ['raw_data'],
        'wind': ['pool_code', 'retry_count', 'request_gap'],
        'fields': ['market_map', 'financial_map'],
        'data_scope': ['start_date', 'end_date']
    }
    
    missing_keys = []
    
    # 1. 递归检查键名是否存在
    for section, keys in required_structure.items():
        if section not in config:
            missing_keys.append(f"缺少主配置节: [{section}]")
            continue
        for key in keys:
            if key not in config[section]:
                missing_keys.append(f"缺少配置项: [{section}] -> {key}")
    
    if missing_keys:
        print("❌ 配置文件校验失败：")
        for error in missing_keys:
            print(f"  - {error}")
        return False

    # 2. 检查路径有效性
    raw_path = config['paths']['raw_data']
    try:
        os.makedirs(raw_path, exist_ok=True)
    except Exception as e:
        print(f"❌ 无法创建数据存储目录: {raw_path}. 错误: {e}")
        return False

    # 3. 检查字段映射是否为空
    if not config['fields']['market_map'] or not config['fields']['financial_map']:
        print("⚠️ 警告: market_map 或 financial_map 为空，这可能导致 wsd 无法获取有效数据。")

    return True

# 执行加载与检查
CONFIG_PATH = "config.yaml"

try:
    with open(CONFIG_PATH, 'r', encoding='utf-8') as f:
        cfg = yaml.safe_load(f)
    
    if validate_config(cfg):
        print(f"✅ config.yaml 读取成功且校验通过")
        print(f"   - 股票池: {cfg['wind']['pool_code']}")
        print(f"   - 时间范围: {cfg['data_scope']['start_date']} 至 {cfg['data_scope']['end_date']}")
    else:
        raise ValueError("配置文件格式不正确，请检查上述错误提示。")

except FileNotFoundError:
    print(f"❌ 错误: 在当前目录下未找到 {CONFIG_PATH}")
except Exception as e:
    print(f"❌ 加载配置文件时发生未知错误: {e}")

✅ config.yaml 读取成功且校验通过
   - 股票池: 1000002
   - 时间范围: 2017-01-01 至 2023-12-31


In [6]:
class WindDataFetcher:
    def __init__(self, config):
        self.cfg = config
        self.raw_path = self.cfg['paths']['raw_data']
        os.makedirs(self.raw_path, exist_ok=True)
        
        if not w.isconnected():
            res = w.start()
            if res.ErrorCode != 0:
                raise RuntimeError(f"Wind API 启动失败: {res}")
        print("✅ Wind API 已连接")

    def get_stock_list(self):
        """获取股票池列表"""
        date_str = datetime.now().strftime("%Y-%m-%d")
        sector_id = self.cfg['wind']['pool_code']
        # wset 用于获取板块成分
        error_code, data = w.wset("sectorconstituent", f"date={date_str};sectorid={sector_id}", usedf=True)
        if error_code != 0:
            raise Exception(f"获取股票列表失败，错误码: {error_code}")
        return data['wind_code'].tolist()

    def _safe_wsd(self, code, fields, start, end, options):
        """带重试机制的 wsd 调用"""
        retry = self.cfg['wind'].get('retry_count', 3)
        for i in range(retry):
            try:
                # wsd 获取历史序列数据
                error_code, data = w.wsd(code, fields, start, end, options, usedf=True)
                if error_code == 0:
                    return data
                elif error_code == -40520007: # 无数据
                    return None
                time.sleep(0.5)
            except Exception as e:
                print(f"⚠️ {code} 请求异常: {e}")
                time.sleep(0.5)
        return None

    def _rename_cols(self, df, mapper):
        """统一字段重命名"""
        df.columns = [mapper.get(c.upper(), c.lower()) for c in df.columns]
        return df

    def fetch_single_stock(self, code, start_date, end_date):
        """核心逻辑：抓取单只股票的行情与财务数据并对齐"""
        mkt_map = self.cfg['fields']['market_map']
        fin_map = self.cfg['fields']['financial_map']
        
        # 1. 行情数据 (日频)
        df_mkt = self._safe_wsd(code, ",".join(mkt_map.keys()), start_date, end_date, "priceAdj=U")
        if df_mkt is None: return None
        df_mkt = self._rename_cols(df_mkt, mkt_map)

        # 2. 财务数据 (季频)
        start_dt = pd.to_datetime(start_date)
        fin_start_str = (start_dt - timedelta(days=180)).strftime("%Y-%m-%d")
        df_fin = self._safe_wsd(code, ",".join(fin_map.keys()), fin_start_str, end_date, "period=Q;unit=1")
        
        if df_fin is not None:
            df_fin = self._rename_cols(df_fin, fin_map)
            df_fin.index = pd.to_datetime(df_fin.index)
            # 以行情时间为基准进行左连接并前向填充
            df_final = df_mkt.join(df_fin, how='left').ffill()
        else:
            df_final = df_mkt
            for col in fin_map.values(): df_final[col] = 0.0

        df_final = df_final.fillna(0.0).reset_index()
        df_final.rename(columns={'index': 'date'}, inplace=True)
        df_final['instrument'] = code
        return df_final

In [7]:
# 初始化抓取器
fetcher = WindDataFetcher(cfg)

# 测试单只股票：检查数据对齐和财务字段
test_stock = "600000.SH" 
df_test = fetcher.fetch_single_stock(
    test_stock, 
    cfg['data_scope']['start_date'], 
    cfg['data_scope']['end_date']
)

if df_test is not None:
    print(f"测试成功：{test_stock}")
    display(df_test.head()) # 展示前几行数据
    # 验证财务字段是否成功填充
    non_zero_fin = (df_test[list(cfg['fields']['financial_map'].values())] != 0).sum().sum()
    print(f"有效财务数据单元格数: {non_zero_fin}")

✅ Wind API 已连接
测试成功：600000.SH


C:\Users\caili\AppData\Local\Temp\ipykernel_35644\3782400039.py:64: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final = df_mkt.join(df_fin, how='left').ffill()


,date,open,high,low,close,volume,amount,adj_factor,vwap,turnover,mkt_cap,earnings,revenue,total_assets,op_cash_flow,total_equity,instrument
0,2018-01-02,12.61,12.77,12.60,12.72,31323053.0,398614966.0,12.201452,12.725930,0.111455,3.733585e+11,0.0,0.0,0.0,0.0,0.0,600000.SH
1,2018-01-03,12.73,12.80,12.66,12.66,37839101.0,480954809.0,12.201452,12.710524,0.134641,3.715973e+11,0.0,0.0,0.0,0.0,0.0,600000.SH
2,2018-01-04,12.70,12.73,12.62,12.66,27883804.0,353205838.0,12.201452,12.667061,0.099217,3.715973e+11,0.0,0.0,0.0,0.0,0.0,600000.SH
3,2018-01-05,12.67,12.71,12.62,12.69,31026744.0,393058250.0,12.201452,12.668369,0.110401,3.724779e+11,0.0,0.0,0.0,0.0,0.0,600000.SH
4,2018-01-08,12.69,12.71,12.63,12.68,31389887.0,397842209.0,12.201452,12.674216,0.111693,3.721844e+11,0.0,0.0,0.0,0.0,0.0,600000.SH


有效财务数据单元格数: 0


In [11]:
# 手动测试 Wind 原始返回

code = "600000.SH"
fields = "tot_oper_rev,net_profit_is" # 确保 config.yaml 里也是这些
# 显式加入 rptType=1 
res = w.wsd(code, fields, "2023-01-01", "2023-12-31", "period=Q;unit=1;rptType=1", usedf=True)

if res[0] != 0:
    print(f"Wind 返回错误，错误码: {res[0]}")
else:
    display(res[1]) # 查看 Dataframe 是否有值

,TOT_OPER_REV,NET_PROFIT_IS
2023-03-31,4.807900e+10,1.615000e+10
2023-06-30,9.123000e+10,2.361500e+10
2023-09-28,NaN,NaN
2023-12-29,NaN,NaN


In [ ]:
stocks = fetcher.get_stock_list()
start_date = cfg['data_scope']['start_date']
end_date = cfg['data_scope']['end_date']

print(f"🚀 开始执行流水线，目标股票数量: {len(stocks)}")

for stock in tqdm(stocks, desc="数据抓取进度"):
    save_path = os.path.join(fetcher.raw_path, f"{stock}.parquet")
    
    # 断点续传逻辑
    if os.path.exists(save_path):
        continue
        
    df = fetcher.fetch_single_stock(stock, start_date, end_date)
    if df is not None:
        # 存储为压缩后的 Parquet 格式
        df.to_parquet(save_path, engine='pyarrow', compression='snappy')
    
    # 避免请求过快被限流
    time.sleep(cfg['wind'].get('request_gap', 0.1))

print("✅ 所有数据抓取完成并存入本地金融仓库。")